In [1]:
import numpy as np
from symmer.symplectic import PauliwordOp

In [61]:
N_qubits_fixed = 5
n_qubits = N_qubits_fixed
binary_vec = (((np.arange(2 ** N_qubits_fixed).reshape([-1, 1]) & (1 << np.arange(N_qubits_fixed))[
                                                                          ::-1])) > 0).astype(bool)

In [55]:
def get_ij_operator(i,j, n_qubits):
    left  = np.array([int(i) for i in np.binary_repr(i, width=n_qubits)]).astype(bool)
    right = np.array([int(i) for i in np.binary_repr(j, width=n_qubits)]).astype(bool)

    AND = left & right # AND where -1 sign
    XOR = left ^ right # XOR where +-i phase

    XZ_mult = left & binary_vec
    ZX_mult = binary_vec & right

    XZX_sign_flips = (-1) ** np.sum(AND & binary_vec, axis=1) # XZX = -X multiplications
    XZ_phase = (-1j) ** np.sum(XZ_mult & ~ZX_mult, axis=1) # XZ=-iY multiplications
    ZX_phase = (+1j) ** np.sum(ZX_mult & ~XZ_mult, axis=1) # ZX=+iY multiplications
    phase_mod = XZX_sign_flips * XZ_phase * ZX_phase
    
    ij_symp_matrix = np.hstack([np.tile(XOR, [2**n_qubits, 1]), binary_vec])
    ij_operator= PauliwordOp(ij_symp_matrix, phase_mod/2**n_qubits)
    
    return ij_operator

In [58]:
op = PauliwordOp.random(n_qubits,15)

matrix = op.to_sparse_matrix.toarray()
row, col = np.where(matrix)

print(len(np.where(matrix)[0]))

print(2**(1.5*n_qubits))

3584
4096.0


In [59]:
%%timeit

P_out = PauliwordOp.empty(n_qubits)
for i,j in zip(row, col):
    ij_op = get_ij_operator(i,j, n_qubits) 
    P_out += ij_op * matrix[i,j]

2.51 s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%timeit PauliwordOp.from_matrix(matrix, strategy='projector')

3.56 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
